you notice a collection of geodes around the pond. Perhaps you could use the obsidian to create some geode-cracking robots and break them open?

To collect the obsidian from the bottom of the pond, you'll need waterproof obsidian-collecting robots. Fortunately, there is an abundant amount of clay nearby that you can use to make them waterproof.

In order to harvest the clay, you'll need special-purpose clay-collecting robots. To make any type of robot, you'll need ore, which is also plentiful but in the opposite direction from the clay.

Collecting ore requires ore-collecting robots with big drills. Fortunately, you have exactly one ore-collecting robot in your pack that you can use to kickstart the whole operation.

Each robot can collect 1 of its resource type per minute. It also takes one minute for the robot factory (also conveniently from your pack) to construct any type of robot, although it consumes the necessary resources available when construction begins.

The robot factory has many blueprints (your puzzle input) you can choose from, but once you've configured it with a blueprint, you can't change it. You'll need to work out which blueprint is best.

You need to figure out which blueprint would maximize the number of opened geodes after 24 minutes by figuring out which robots to build and when to build them.

Blueprint 1:<br>
  Each ore robot costs 4 ore.<br>
  Each clay robot costs 2 ore.<br>
  Each obsidian robot costs 3 ore and 14 clay.<br>
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:<br>
  Each ore robot costs 2 ore.<br>
  Each clay robot costs 3 ore.<br>
  Each obsidian robot costs 3 ore and 8 clay.<br>
  Each geode robot costs 3 ore and 12 obsidian.

I wanna try creating a treelike hierachy, where each node represents a minute and what robots are available, what ressources and whether or not to build one robot or another. At each minute a decision is to be made, whether there are ressources enough to build a robot and which robot to build, but also to not build a robot and instead wait to gather more ressources to build another robot.

It only takes a minute to build a robot, so in the steps adding the robot material production will just have to be added in the end, then I wont have to do any queuing.

- Pop first entry of list
- Determine whether it is minute 24
- Collect materials, if minute 24 save the path
- Check what robots can be build
- Append a new entry for each possible decision, where the path is the robots build and update the material production
- Pop next entry

I am going to do a breadth-first search of the network to build the network, this way it will build one layer at a time, and because each layer is one minute, it stops before starting the 25th minute.

Each node needs to have an id, the path, the minutes, the robots producing material and the current material.

In [2]:
import copy
import re

In [37]:
class RobotNode():
    def __init__(self):
        self.path = []
        self.min = 0
        self.robots_factory = []
        self.material_production = {"ore":0,
                                    "clay":0,
                                    "obsidian":0,
                                    "geode":0,
                                   }
        self.current_ratio = 0
        self.material = {"ore":0,
                         "clay":0,
                         "obsidian":0,
                         "geode":0
                         }
        
    def update_production(self,robot):
        self.material_production[robot] += 1
        
        # Update current ratio
        ore_amount = self.material_production["ore"]

        clay_amount = self.material_production["clay"]

        try:
            self.current_ratio = (ore_amount*self.ore_cost)/(clay_amount*self.clay_cost)
        except ZeroDivisionError:
            self.current_ratio = 1
        return
    
    def update_material(self):
        for rock, production in self.material_production.items():
            self.material[rock] += production
        return
    
    def get_blueprints(self, blueprints):
        self.blueprints = blueprints
        self.ore_amount = self.blueprints["obsidian"]["ore"]
        self.ore_cost = self.blueprints["ore"]["ore"]

        self.clay_amount = self.blueprints["obsidian"]["clay"]
        self.clay_cost = self.blueprints["clay"]["ore"] * self.ore_cost

        self.ideal_ratio = (self.ore_amount*self.ore_cost)/(self.clay_amount*self.clay_cost)

        # If the ratio of ore gets below the difference between the ideal ratio and how much it changes by adding one ore, 
        # then we add an extra ore
        extra_ore = ((self.ore_amount+1)*self.ore_cost)/(self.clay_amount*self.clay_cost)
        extra_clay = (self.ore_amount*self.ore_cost)/((self.clay_amount+1)*self.clay_cost)
        self.add_ore = self.ideal_ratio-(extra_ore - self.ideal_ratio)
        self.add_clay = self.ideal_ratio-(extra_clay - self.ideal_ratio) # Not currently implemented
        
    def new_node(self, robot):
        new_node = copy.deepcopy(self)
        new_node.path.extend([robot])
        new_node.min += 1
        new_node.update_material()
        if robot:
            for rock, value in self.blueprints[robot].items():
                new_node.material[rock] -= value
            new_node.update_production(robot)
        return new_node
    
    def does_it_build(self, robot):
        build = True
        # From the blueprints the material stock is checked, to see if there is enough material for a new robot
        for rock, value in self.blueprints[robot].items():
            if value > self.material[rock]:
                build = False
                break
        return build
        
    
    def build_robots(self):
        # Create new list of robots to extend the queue
        update_queue = []
        # Check if geode or obsidian is possible to build, end if geode is possible
        for i, robot in enumerate(["geode","obsidian"]):
            if self.does_it_build(robot):
                new_robotnode = self.new_node(robot)
                update_queue.append(new_robotnode)
                break
            
            # If not possible to build geode or obsidian, check the current ratio, and begin building according to that
            else:
                if self.current_ratio <= self.add_ore:
                    robot = "ore"
                else:
                    robot = "clay"
                
                if self.does_it_build(robot):
                    new_robotnode = self.new_node(robot)
                    update_queue.append(new_robotnode)
                else:
                    new_robotnode = self.new_node(None)
                    update_queue.append(new_robotnode)

        return update_queue

In [6]:
with open("day_19_input.txt", "r") as file:
    text = file.read()

In [7]:
text_split = text.split("\n")
text_split[:4]

['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 14 clay. Each geode robot costs 2 ore and 16 obsidian.',
 'Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 15 clay. Each geode robot costs 2 ore and 7 obsidian.',
 'Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.',
 'Blueprint 4: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 17 clay. Each geode robot costs 3 ore and 16 obsidian.']

In [8]:
blueprints = []
for i, blueprint in enumerate(text_split):
    materials = re.findall("(ore|clay|obsidian|geode)(?:\srobot)(?:\D*)(\d*)(?:\s)(\w*)(?:\sand\s)?(\d*)?(?:\s)?(\w*)?",blueprint)
    blueprint = {}
    for material in materials:
        robot = material[0]
        build_materials = [x for x in material[1:] if re.match("\D", x)]
        build_quantity = [int(x) for x in material[1:] if re.match("\d", x)]
        blueprint[robot] = dict(zip(build_materials,build_quantity))
    blueprints.append(blueprint)

In [464]:
geode_scores_max = []
for i, blueprint in enumerate(blueprints):
    print(i, "/", len(blueprints))
    results = []

    # Defining inital state
    robot_test = RobotNode()
    robot_test.update_production("ore")
    robot_test.get_blueprints(blueprint)

    queue = [robot_test]
    minute = 0

    test_minutes = 24
    best_scores = {"obsidian":{"minute":0, "score":0},"geode":{"minute":0, "score":0}}

    while minute < test_minutes:
        current_node = queue.pop(0)

        minute = current_node.min
        obsidian_score = current_node.material_production["obsidian"]
        geode_score = current_node.material_production["geode"]


        if geode_score <= best_scores["geode"]["score"]:
            # Determine if the obsidian-score is higher or the same - if it is the same, we will look at whether the minutes
            # are lower - did it achieve the same score quicker.
            if obsidian_score >= best_scores["obsidian"]["score"]:
                best_scores["obsidian"]["score"] = obsidian_score
                if (minute < best_scores["obsidian"]["minute"])|(obsidian_score > best_scores["obsidian"]["score"]):
                    best_scores["obsidian"]["minute"] = minute
            elif minute > best_scores["obsidian"]["minute"]:
                continue

        if minute == test_minutes - 1:
            current_node.update_material()
            path = current_node.path
            production_result = current_node.material
            results.append((path,production_result))
        extend_queue = current_node.build_robots()
        if extend_queue == []:
            continue
        queue.extend(extend_queue)
    geode_max = max([node[1]["geode"] for node in results])
    geode_scores_max.append(geode_max)

0 / 31
1 / 31


KeyboardInterrupt: 

In [465]:
blueprints[1]

{'ore': {'ore': 2},
 'clay': {'ore': 2},
 'obsidian': {'ore': 2, 'clay': 15},
 'geode': {'ore': 2, 'obsidian': 7}}

In [31]:
results = []

# Defining inital state
robot_test = RobotNode()
# robot_test.update_production("ore")
blueprints = {"ore":{"ore":4}, 
              "clay":{"ore":2},
              "obsidian":{"ore":3,"clay":14},
              "geode":{"ore":2,"obsidian":7}
             }
robot_test.get_blueprints(blueprints)

In [38]:
results = []

# Defining inital state
robot_test = RobotNode()

blueprints = {"ore":{"ore":4}, 
              "clay":{"ore":2},
              "obsidian":{"ore":3,"clay":14},
              "geode":{"ore":2,"obsidian":7}
             }
robot_test.get_blueprints(blueprints)
robot_test.update_production("ore")
queue = [robot_test]
minute = 0

test_minutes = 24
best_scores = {"obsidian":{"minute":0, "score":0},"geode":{"minute":0, "score":0}}

while minute < test_minutes:
    current_node = queue.pop(0)
    
    minute = current_node.min
    obsidian_score = current_node.material_production["obsidian"]
    geode_score = current_node.material_production["geode"]
    
    
    if geode_score <= best_scores["geode"]["score"]:
        # Determine if the obsidian-score is higher or the same - if it is the same, we will look at whether the minutes
        # are lower - did it achieve the same score quicker.
        if obsidian_score >= best_scores["obsidian"]["score"]:
            best_scores["obsidian"]["score"] = obsidian_score
            if (minute < best_scores["obsidian"]["minute"])|(obsidian_score > best_scores["obsidian"]["score"]):
                best_scores["obsidian"]["minute"] = minute
        elif minute > best_scores["obsidian"]["minute"]:
            continue

    
    if minute == test_minutes - 1:
        current_node.update_material()
        path = current_node.path
        production_result = current_node.material
        results.append((path,production_result))
    extend_queue = current_node.build_robots()
    if extend_queue == []:
        continue
    queue.extend(extend_queue)


KeyboardInterrupt: 

In [ ]:
blueprints = {"ore":{"ore":4}, 
              "clay":{"ore":2},
              "obsidian":{"ore":3,"clay":14},
              "geode":{"ore":2,"obsidian":7}
             }

In [486]:
[node[1] for node in results if (node[1]["geode"] == 9)&(node[1]["obsidian"] == 10)]

[{'ore': 3, 'clay': 27, 'obsidian': 10, 'geode': 9},
 {'ore': 3, 'clay': 26, 'obsidian': 10, 'geode': 9},
 {'ore': 3, 'clay': 26, 'obsidian': 10, 'geode': 9},
 {'ore': 3, 'clay': 26, 'obsidian': 10, 'geode': 9},
 {'ore': 3, 'clay': 26, 'obsidian': 10, 'geode': 9}]

In [478]:
3/27, 3/26

(0.1111111111111111, 0.11538461538461539)

In [488]:
(4*3)/(14*2*4)

0.10714285714285714

In [493]:
blueprints[0]

{'ore': {'ore': 4},
 'clay': {'ore': 4},
 'obsidian': {'ore': 4, 'clay': 14},
 'geode': {'ore': 2, 'obsidian': 16}}

In [9]:
ore_amount = blueprints[0]["obsidian"]["ore"]
ore_cost = blueprints[0]["ore"]["ore"]

clay_amount = blueprints[0]["obsidian"]["clay"]
clay_cost = blueprints[0]["clay"]["ore"] * ore_cost

ideal_ratio = (ore_amount*ore_cost)/(clay_amount*clay_cost)
ideal_ratio

# If the ratio of ore gets below the difference between the ideal ratio and how much it changes by adding one ore, 
# then we add an extra ore
extra_ore = ((ore_amount+1)*ore_cost)/(clay_amount*clay_cost)
extra_clay = (ore_amount*ore_cost)/((clay_amount+1)*clay_cost)
add_ore = ideal_ratio-(extra_ore - ideal_ratio)
add_clay = ideal_ratio-(extra_clay - ideal_ratio)
add_clay

0.07142857142857142

In [22]:
4/14

0.2857142857142857

In [19]:
# If the ratio of ore gets below the difference between the ideal ratio and how much it changes by adding one ore, 
# then we add an extra ore
extra_ore = ((ore_amount+1)*ore_cost)/(clay_amount*clay_cost)
extra_clay = (ore_amount*ore_cost)/((clay_amount+1)*clay_cost)
add_ore = ideal_ratio-(extra_ore - ideal_ratio)
add_clay = ideal_ratio-(extra_clay - ideal_ratio)
add_clay

0.07619047619047618

In [21]:
extra_clay - ideal_ratio

-0.004761904761904759

In [78]:
class RobotNode():
    def __init__(self):
        self.path = []
        self.min = 0
        self.robots_factory = []
        self.material_production = {"ore":0,
                                    "clay":0,
                                    "obsidian":0,
                                    "geode":0,
                                   }
        self.current_ratio = 0
        self.material = {"ore":0,
                         "clay":0,
                         "obsidian":0,
                         "geode":0
                         }
        
    def update_production(self,robot):
        self.material_production[robot] += 1
        
        # Update current ratio
        ore_amount = self.material_production["ore"]

        clay_amount = self.material_production["clay"]

        try:
            self.current_ratio = (ore_amount*self.ore_cost)/(clay_amount*self.clay_cost)
        except ZeroDivisionError:
            self.current_ratio = 1
        return
    
    def update_material(self):
        for rock, production in self.material_production.items():
            self.material[rock] += production
        return
    
    def get_blueprints(self, blueprints):
        self.blueprints = blueprints
        self.ore_amount = self.blueprints["obsidian"]["ore"]
        self.ore_cost = self.blueprints["ore"]["ore"]

        self.clay_amount = self.blueprints["obsidian"]["clay"]
        self.clay_cost = self.blueprints["clay"]["ore"] * self.ore_cost

        self.ideal_ratio = (self.ore_amount*self.ore_cost)/(self.clay_amount*self.clay_cost)

        # If the ratio of ore gets below the difference between the ideal ratio and how much it changes by adding one ore, 
        # then we add an extra ore
        extra_ore = ((self.ore_amount+1)*self.ore_cost)/(self.clay_amount*self.clay_cost)
        extra_clay = (self.ore_amount*self.ore_cost)/((self.clay_amount+1)*self.clay_cost)
        self.add_ore = self.ideal_ratio-(extra_ore - self.ideal_ratio)
        self.add_clay = self.ideal_ratio-(extra_clay - self.ideal_ratio) # Not currently implemented
        
    def new_node(self, robot):
        new_node = copy.deepcopy(self)
        new_node.path.extend([robot])
        new_node.min += 1
        new_node.update_material()
        if robot:
            for rock, value in self.blueprints[robot].items():
                new_node.material[rock] -= value
            new_node.update_production(robot)
        return new_node
    
    def does_it_build(self, robot):
        build = True
        # From the blueprints the material stock is checked, to see if there is enough material for a new robot
        for rock, value in self.blueprints[robot].items():
            if value > self.material[rock]:
                build = False
                break
        return build
        
    
    def build_robots(self):
        # Check if geode or obsidian is possible to build, end if geode is possible
        for i, robot in enumerate(["geode","obsidian"]):
            if self.does_it_build(robot):
                break
            
            # If not possible to build geode or obsidian, check the current ratio, and begin building according to that
            else:
                if abs(self.current_ratio - min(self.add_ore,self.add_clay)) <= min(self.add_ore,self.add_clay):
                    robot = None
                else:
                    if self.current_ratio <= self.add_ore:
                        robot = "ore"
                    else:
                        robot = "clay"

                    if self.does_it_build(robot) == False:
                        robot = None
        
        if robot:
            for rock, value in self.blueprints[robot].items():
                self.material[rock] -= value
    
        return robot

In [50]:
robot_test.update_material()
robot_test.material

{'ore': 9, 'clay': 0, 'obsidian': 0, 'geode': 0}

In [51]:
robot_test.build_robots()

'clay'

Each ore robot costs 2 ore.
Each clay robot costs 3 ore.
Each obsidian robot costs 3 ore and 8 clay.
Each geode robot costs 3 ore and 12 obsidian.

In [90]:
# Defining inital state
robot_test = RobotNode()

blueprints = {"ore":{"ore":2}, 
              "clay":{"ore":3},
              "obsidian":{"ore":3,"clay":8},
              "geode":{"ore":3,"obsidian":12}
             }
robot_test.get_blueprints(blueprints)
robot_test.update_production("ore")

build_path = []
for i in range(24):
    robot_test.update_material()
    build = robot_test.build_robots()
    if build:
        robot_test.update_production(build)
    build_path.append(build)
robot_test.update_material()

In [91]:
robot_test.material

{'ore': 1, 'clay': 9, 'obsidian': 15, 'geode': 9}

In [92]:
build_path

[None,
 None,
 'clay',
 None,
 None,
 'clay',
 None,
 None,
 'obsidian',
 None,
 None,
 None,
 'obsidian',
 None,
 None,
 None,
 'geode',
 'obsidian',
 None,
 None,
 'obsidian',
 None,
 None,
 'geode']